In [ ]:
!pip install gdown

import gdown
import os

# Скачиваем архивы моделей
t5_file_id = "1PjlnG723Ug53RmWocgUOzqnIk1mPjqVB"  # T5 модель
roberta_file_id = "1PSEUte091ouNFaxuIuLYqaAnIaAAOCBi"  # RoBERTa модель

# Выходные пути
t5_output = "t5_model.zip"
roberta_output = "roberta_model.zip"

# Скачивание
gdown.download(f"https://drive.google.com/uc?id={t5_file_id}", t5_output, quiet=False)
gdown.download(f"https://drive.google.com/uc?id={roberta_file_id}", roberta_output, quiet=False)

# Распаковка архивов
!unzip -o t5_model.zip -d ./t5_model
!unzip -o roberta_model.zip -d ./roberta_model


In [ ]:
!pip install --upgrade transformers
!pip install timm

from transformers import AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration

# ---------------------------
# 1) Configure Device
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Путь к T5
t5_path = "./t5_model"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_path)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_path).to(device)

# Путь к RoBERTa
roberta_path = "./roberta_model"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_path)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_path).to(device)

print(f"T5 Model Device: {next(t5_model.parameters()).device}")
print(f"RoBERTa Model Device: {next(roberta_model.parameters()).device}")

# BART for summarization
summarizer_pipeline = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)


In [ ]:
import random

questions = [
    # SUPPORTS
    {"question": "What is the capital of France?", "type": "fact", "statement": "The capital of France is Paris.", "label": "SUPPORTS"},
    {"question": "Who painted the Mona Lisa?", "type": "fact", "statement": "The Mona Lisa was painted by Leonardo da Vinci.", "label": "SUPPORTS"},
    {"question": "What is the chemical formula for water?", "type": "fact", "statement": "The chemical formula for water is H2O.", "label": "SUPPORTS"},
    {"question": "What is the smallest planet in our solar system?", "type": "fact", "statement": "The smallest planet in our solar system is Mercury.", "label": "SUPPORTS"},
    {"question": "Who wrote '1984'?", "type": "fact", "statement": "The book '1984' was written by George Orwell.", "label": "SUPPORTS"},
    {"question": "What is the tallest mountain in the world?", "type": "fact", "statement": "The tallest mountain in the world is Mount Everest.", "label": "SUPPORTS"},
    {"question": "Who was the first man on the moon?", "type": "fact", "statement": "The first man on the moon was Neil Armstrong.", "label": "SUPPORTS"},
    {"question": "What is the capital of Japan?", "type": "fact", "statement": "The capital of Japan is Tokyo.", "label": "SUPPORTS"},
    {"question": "What is the primary gas in Earth's atmosphere?", "type": "fact", "statement": "The primary gas in Earth's atmosphere is nitrogen.", "label": "SUPPORTS"},
    {"question": "Who discovered penicillin?", "type": "fact", "statement": "Penicillin was discovered by Alexander Fleming.", "label": "SUPPORTS"},
    {"question": "What is the most abundant element in the universe?", "type": "fact", "statement": "The most abundant element in the universe is hydrogen.", "label": "SUPPORTS"},
    {"question": "What is the capital of Russia?", "type": "fact", "statement": "The capital of Russia is Moscow.", "label": "SUPPORTS"},
    {"question": "Who wrote 'The Great Gatsby'?", "type": "fact", "statement": "The book 'The Great Gatsby' was written by F. Scott Fitzgerald.", "label": "SUPPORTS"},
    {"question": "What is the smallest bone in the human body?", "type": "fact", "statement": "The smallest bone in the human body is the stapes.", "label": "SUPPORTS"},
    {"question": "What is the capital of Spain?", "type": "fact", "statement": "The capital of Spain is Madrid.", "label": "SUPPORTS"},
    {"question": "Who is known as the father of modern physics?", "type": "fact", "statement": "The father of modern physics is Albert Einstein.", "label": "SUPPORTS"},
    {"question": "What is the deepest ocean trench in the world?", "type": "fact", "statement": "The deepest ocean trench in the world is the Mariana Trench.", "label": "SUPPORTS"},
    {"question": "What is the largest continent on Earth?", "type": "fact", "statement": "The largest continent on Earth is Asia.", "label": "SUPPORTS"},
    {"question": "What is the hottest planet in our solar system?", "type": "fact", "statement": "The hottest planet in our solar system is Venus.", "label": "SUPPORTS"},

    # REFUTES
    {"question": "What is the capital of Italy?", "type": "fact", "statement": "The capital of Italy is Venice.", "label": "REFUTES"},
    {"question": "Who discovered electricity?", "type": "fact", "statement": "Electricity was discovered by Nikola Tesla.", "label": "REFUTES"},
    {"question": "What is the boiling point of water?", "type": "fact", "statement": "The boiling point of water is 150 degrees Celsius.", "label": "REFUTES"},
    {"question": "What is the atomic number of hydrogen?", "type": "fact", "statement": "The atomic number of hydrogen is 10.", "label": "REFUTES"},
    {"question": "What is the largest desert in the world?", "type": "fact", "statement": "The largest desert in the world is the Amazon Rainforest.", "label": "REFUTES"},
    {"question": "What is the capital of Canada?", "type": "fact", "statement": "The capital of Canada is Toronto.", "label": "REFUTES"},
    {"question": "Who was the first president of the United States?", "type": "fact", "statement": "The first president of the United States was Abraham Lincoln.", "label": "REFUTES"},
    {"question": "What is the speed of light?", "type": "fact", "statement": "The speed of light is 400,000 kilometers per second.", "label": "REFUTES"},
    {"question": "What is the largest ocean on Earth?", "type": "fact", "statement": "The largest ocean on Earth is the Atlantic.", "label": "REFUTES"},
    {"question": "What is the primary language spoken in Brazil?", "type": "fact", "statement": "The primary language spoken in Brazil is Spanish.", "label": "REFUTES"},
    {"question": "What is the capital of Germany?", "type": "fact", "statement": "The capital of Germany is Munich.", "label": "REFUTES"},
    {"question": "Who invented the airplane?", "type": "fact", "statement": "The airplane was invented by Nikola Tesla.", "label": "REFUTES"},
    {"question": "What is the currency of Japan?", "type": "fact", "statement": "The currency of Japan is the US Dollar.", "label": "REFUTES"},
    {"question": "What is the freezing point of water?", "type": "fact", "statement": "The freezing point of water is -50 degrees Celsius.", "label": "REFUTES"},
    {"question": "What is the largest city in Australia?", "type": "fact", "statement": "The largest city in Australia is Canberra.", "label": "REFUTES"},
    {"question": "What is the smallest country in the world?", "type": "fact", "statement": "The smallest country in the world is Monaco.", "label": "REFUTES"},
    {"question": "Who painted 'Starry Night'?", "type": "fact", "statement": "'Starry Night' was painted by Pablo Picasso.", "label": "REFUTES"},
    {"question": "What is the square root of 64?", "type": "fact", "statement": "The square root of 64 is 10.", "label": "REFUTES"},
    {"question": "Who discovered gravity?", "type": "fact", "statement": "Gravity was discovered by Galileo Galilei.", "label": "REFUTES"},

    # NOT ENOUGH INFO
    {"question": "What is the longest river in the world?", "type": "fact", "statement": "The longest river in the world is the Nile.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the population of Antarctica?", "type": "fact", "statement": "The population of Antarctica is 10,000.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the tallest tree species in the world?", "type": "fact", "statement": "The tallest tree species in the world is the Baobab.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the temperature on Venus?", "type": "fact", "statement": "The temperature on Venus is 500 degrees Celsius.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the national animal of Canada?", "type": "fact", "statement": "The national animal of Canada is the moose.", "label": "NOT ENOUGH INFO"},
    {"question": "Who was the first human in space?", "type": "fact", "statement": "The first human in space was Alan Shepard.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the chemical symbol for gold?", "type": "fact", "statement": "The chemical symbol for gold is Au.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the capital of South Africa?", "type": "fact", "statement": "The capital of South Africa is Johannesburg.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the largest animal ever recorded?", "type": "fact", "statement": "The largest animal ever recorded is the Megalodon.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the circumference of the Earth?", "type": "fact", "statement": "The circumference of the Earth is 40,000 miles.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the most common element in the universe?", "type": "fact", "statement": "The most common element in the universe is helium.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the distance between the Earth and the Moon?", "type": "fact", "statement": "The distance between the Earth and the Moon is 384,000 kilometers.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the highest recorded temperature on Earth?", "type": "fact", "statement": "The highest recorded temperature on Earth is 58 degrees Celsius.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the oldest book ever written?", "type": "fact", "statement": "The oldest book ever written is 'The Epic of Gilgamesh'.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the origin of the universe?", "type": "concept", "statement": "The origin of the universe is the Big Bang theory.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the function of the appendix?", "type": "concept", "statement": "The function of the appendix is to store good bacteria.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the main ingredient in chocolate?", "type": "fact", "statement": "The main ingredient in chocolate is vanilla.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the speed of sound in water?", "type": "fact", "statement": "The speed of sound in water is 1,500 meters per second.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the primary purpose of photosynthesis?", "type": "concept", "statement": "The primary purpose of photosynthesis is to produce oxygen.", "label": "NOT ENOUGH INFO"},
    {"question": "What is the capital of Australia?", "type": "fact", "statement": "The capital of Australia is Melbourne.", "label": "NOT ENOUGH INFO"}

     # Противоречивые
    {"question": "Is light a wave or a particle?", "type": "concept", "statement": "Light is a wave.", "label": "REFUTES"},
    {"question": "Is Pluto a planet?", "type": "fact", "statement": "Pluto is classified as a planet.", "label": "REFUTES"},
    {"question": "Did humans evolve solely from Homo habilis?", "type": "concept", "statement": "Humans evolved solely from Homo habilis.", "label": "REFUTES"},
    {"question": "Can AI develop consciousness?", "type": "concept", "statement": "Artificial intelligence can develop self-awareness.", "label": "NOT ENOUGH INFO"},

    # Специализированные
    {"question": "What is the Heisenberg uncertainty principle?", "type": "concept", "statement": "The Heisenberg uncertainty principle states that the position and momentum of a particle can be precisely measured simultaneously.", "label": "REFUTES"},
    {"question": "What is the primary function of the hippocampus?", "type": "fact", "statement": "The hippocampus is responsible for motor control.", "label": "REFUTES"},
    {"question": "What is the Schwarzschild radius?", "type": "concept", "statement": "The Schwarzschild radius is the minimum distance at which an object must orbit the Earth to avoid being pulled into the atmosphere.", "label": "REFUTES"},
    {"question": "Does quantum entanglement allow for faster-than-light communication?", "type": "concept", "statement": "Quantum entanglement enables instantaneous communication over long distances.", "label": "REFUTES"},

    # Дискуссионные/философские
    {"question": "Can mathematics fully describe reality?", "type": "concept", "statement": "Mathematics is a complete and perfect language for describing all aspects of reality.", "label": "NOT ENOUGH INFO"},
    {"question": "Is morality absolute or relative?", "type": "concept", "statement": "Moral values are absolute and universally applicable.", "label": "NOT ENOUGH INFO"},
    {"question": "Is time an absolute concept?", "type": "concept", "statement": "Time is absolute and independent of any observer.", "label": "REFUTES"},
    {"question": "Can the universe be infinite?", "type": "concept", "statement": "The universe has a finite boundary.", "label": "NOT ENOUGH INFO"}
]

# Shuffle the dataset
random.shuffle(questions)


In [ ]:
# Тест T5
question = "What is the capital of France?"
input_text = f"Question: {question}\nAnswer briefly."
t5_inputs = t5_tokenizer(input_text, return_tensors="pt").to(device) # Move t5_inputs to the device
t5_outputs = t5_model.generate(**t5_inputs)
print("T5 Output:", t5_tokenizer.decode(t5_outputs[0], skip_special_tokens=True))

# Тест RoBERTa
statement = "The capital of France is Paris."
context = "Paris is the capital and most populous city of France."
roberta_inputs = roberta_tokenizer(f"Statement: {statement} Context: {context}", return_tensors="pt").to(device) # Move roberta_inputs to the device
roberta_outputs = roberta_model(**roberta_inputs)
logits = roberta_outputs.logits
print("RoBERTa Output:", logits)


In [ ]:
# Install required libraries
!pip install --upgrade --quiet transformers google-api-python-client

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration
from googleapiclient.discovery import build
from tqdm import tqdm
import gspread
from google.colab import auth
from google.auth import default

# ---------------------------
# 1) Configure Device
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# 2) Models Already Loaded
# ---------------------------
# Assuming models were loaded in a previous step
# Variables to use:
# - roberta_tokenizer
# - roberta_model
# - t5_tokenizer
# - t5_model
# - summarizer_pipeline

# ---------------------------
# 3) Google Custom Search API
# ---------------------------
def retrieve_context_from_google(question, top_k=3):
    """
    Use Google Custom Search API to retrieve top_k results for the given question.
    Returns concatenated snippets of the search results.
    """
    api_key = ""  # Replace with your actual API key
    cse_id = ""  # Replace with your Custom Search Engine ID
    service = build("customsearch", "v1", developerKey=api_key)

    try:
        result = service.cse().list(q=question, cx=cse_id, num=top_k).execute()
        snippets = [item.get("snippet", "") for item in result.get("items", [])]
        return " ".join(snippets) if snippets else "No relevant Google context found."
    except Exception as e:
        print(f"Error retrieving context from Google: {e}")
        return "No relevant Google context found."

# ---------------------------
# 4) Helper Functions
# ---------------------------
def summarize_context(context, max_len=150, min_len=40):
    """
    Summarizes the context text using BART. Adjust max_len/min_len as needed.
    """
    if not context.strip():
        return "No context to summarize."
    summarized = summarizer_pipeline(context, max_length=max_len, min_length=min_len, do_sample=False)
    return summarized[0]['summary_text']

def ask_if_context_needed(question):
    """
    Use T5 to decide whether external context is required to answer the given question.
    Returns:
        - Boolean (True if context is needed, False otherwise)
        - Raw response from the model (reasoning chain)
    """
    prompt = (
        "You are a reasoning assistant. Decide if the question below requires external knowledge "
        "like a search engine or is answerable by common knowledge and simple logic alone.\n\n"
        "INSTRUCTIONS:\n"
        "1. Think step by step through the question.\n"
        "2. If you are NOT 100% certain you can answer without external knowledge, say YES.\n"
        "3. If you are certain it is trivial or well-known, say NO.\n\n"
        f"Question: {question}\n\n"
        "Let's reason step by step and conclude with either 'YES' or 'NO':"
    )

    inputs = t5_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = t5_model.generate(inputs["input_ids"], max_length=100, num_beams=4, early_stopping=True)
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Decide based on the response
    if "YES" in response.upper():
        return True, response
    elif "NO" in response.upper():
        return False, response
    else:
        # Default to YES if ambiguous
        return True, response


def classify_with_roberta(statement, context):
    """
    Classify a statement as SUPPORTS, REFUTES, or NOT ENOUGH INFO based on context.
    """
    prompt = f"Statement: {statement}\nContext: {context}\nClassify into SUPPORTS, REFUTES, or NOT ENOUGH INFO."
    inputs = roberta_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = roberta_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    confidence, label_idx = torch.max(probabilities, dim=1)
    labels = ["REFUTES", "NOT ENOUGH INFO", "SUPPORTS"]
    return labels[label_idx.item()], float(confidence.item())

def generate_with_t5(question, context):
    """
    Generate an answer with T5 for a given question and context. If you aren't confident, say "I don't know."
    """
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer briefly."
    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = t5_model.generate(inputs["input_ids"], max_length=64, num_beams=4)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

# ---------------------------
# 5) Google Sheets Setup
# ---------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1VDOKkz09sVWTBGtcD9Ne654o2OtumEyIqBIOELGBWW0/edit?gid=0#gid=0").sheet1


# ---------------------------
# 6) Example Dataset (50 Questions)
# ---------------------------


def self_assess_with_t5(question, context, generated_answer):
    """
    Use T5 to self-assess confidence in its generated answer.
    Returns a score from 1 to 100.
    """
    prompt = (
        "You answered the following question. Assess your confidence in the answer on a scale from 1 to 5.\n"
        f"Question: {question}\n"
        f"Context: {context}\n"
        f"Your Answer: {generated_answer}\n"
        "Confidence (1-5):"
    )
    inputs = t5_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = t5_model.generate(inputs["input_ids"], max_length=20, num_beams=4, early_stopping=True)
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract numerical confidence
    try:
        confidence_score = int("".join(filter(str.isdigit, response)))
        return confidence_score if 1 <= confidence_score <= 5 else 1
    except ValueError:
        return 1


def roberta_self_assess(statement, context):
    """
    Use RoBERTa to self-assess confidence in its classification result.
    Returns a self-assessed confidence score from 1 to 100.
    """
    prompt = (
        f"Statement: {statement}\n"
        f"Context: {context}\n"
        "I just classified this statement. Now, self-assess your confidence in your own classification.\n"
        "Provide a confidence score on a scale from 1 to 5 based on how well the context supports your decision:"
    )
    inputs = roberta_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = roberta_model(**inputs)
    logits = outputs.logits

    # Extract confidence directly from logits for simplicity
    probabilities = torch.softmax(logits, dim=-1)
    max_prob = probabilities.max().item()

    # Scale the max_prob (0-1) to a confidence score (1-5)
    self_assessment_score = int(max_prob * 5)
    return self_assessment_score


import torch.nn.functional as F

def get_t5_confidence(question, context):
    """
    Calculate T5 confidence for the generated answer based on output logits.
    Returns the generated answer and confidence score.
    """
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer briefly."
    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Generate output while retaining logits
    outputs = t5_model.generate(
        inputs["input_ids"],
        max_length=64,
        num_beams=4,
        return_dict_in_generate=True,
        output_scores=True
    )

    generated_ids = outputs.sequences[0]
    token_scores = outputs.scores  # List of logits for each step in the output sequence

    # Decode the answer
    generated_answer = t5_tokenizer.decode(generated_ids, skip_special_tokens=True)

    # Safeguard against mismatched lengths
    token_probs = []
    for step, logits in enumerate(token_scores[:len(generated_ids) - 1]):  # Ensure alignment
        token_id = generated_ids[step + 1]  # Offset by 1 because scores are for the next token
        token_prob = F.softmax(logits, dim=-1)[0, token_id].item()
        token_probs.append(token_prob)

    # Calculate the average confidence over all tokens
    avg_confidence = sum(token_probs) / len(token_probs) if token_probs else 0.0
    return generated_answer, avg_confidence


# ---------------------------
# 7) Main Pipeline
# ---------------------------
results = []

for q in tqdm(questions, desc="Processing Questions"):
    question = q["question"]
    statement = q["statement"]
    original_label = q["label"]

    if original_label == "NOT ENOUGH INFO":
      context = ""
      summarized_context = ""
    else:
      context = retrieve_context_from_google(question)
      summarized_context = summarize_context(context)

    # Step 2: Ask if context is needed for text generation (T5)
    needs_context, reasoning = ask_if_context_needed(question)

    if not needs_context:
        # If T5 doesn't need context, ignore retrieved context for answer generation
        t5_context = "No external context used."
        t5_summarized_context = "No summary (context not needed)."
    else:
        t5_context = context
        t5_summarized_context = summarized_context

    # Step 3: Generate answer with T5
    generated_answer, t5_confidence = get_t5_confidence(question, t5_summarized_context)

    # Step 4: Perform self-assessment with T5
    t5_self_assessment = self_assess_with_t5(question, t5_summarized_context, generated_answer)

    # Step 5: Classify the statement with RoBERTa
    label, roberta_confidence = classify_with_roberta(statement, summarized_context)

    roberta_self_assessment = roberta_self_assess(statement, summarized_context)


    # Step 6: Append results
    results.append([
        question,
        statement,
        original_label,  # Original label from dataset
        needs_context,
        reasoning,
        t5_context,
        t5_summarized_context,
        context,
        summarized_context,
        generated_answer,
        t5_confidence,  # T5's confidence in its answer
        t5_self_assessment,  # T5's self-assessment score
        label,
        roberta_confidence,
        roberta_self_assessment  # Self-assessment from RoBERTa
    ])

# ---------------------------
# 8) Write to Google Sheet
# ---------------------------
headers = [
    "Question",
    "Statement",
    "Original Label",
    "Needs Context?",
    "Context Decision Reasoning",
    "T5 Context",
    "T5 Summarized Context",
    "Roberta Context",
    "Roberta Summarized Context",
    "T5 Generated Answer",
    "T5 Confidence",
    "T5 Self-Assessment",  # New column for self-assessment score
    "Classification",
    "Roberta Confidence",
    "Roberta Self-Assessment"
]

sheet.clear()
sheet.append_row(headers)

for row in results:
    sheet.append_row(row)

print("Results written to Google Sheet!")


In [ ]:
# Install necessary libraries
!pip install --quiet matplotlib pandas gspread oauth2client seaborn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import auth
import gspread
from google.auth import default

# ---------------------------
# 1) Authenticate and Load Data
# ---------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Replace with your Google Sheet URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1VDOKkz09sVWTBGtcD9Ne654o2OtumEyIqBIOELGBWW0/edit?gid=0#gid=0'
sheet = gc.open_by_url(sheet_url).sheet1

# Load data into a Pandas DataFrame
data = pd.DataFrame(sheet.get_all_records())

# Helper function to clean data
def clean_data(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df.dropna(subset=columns)

# ---------------------------
# 2) Visualizations
# ---------------------------

# Visualization 1: Distribution of Classification Labels
plt.figure(figsize=(8, 6))
data["Classification"].value_counts().plot(kind="bar", color=["skyblue", "salmon", "lime"])
plt.title("Distribution of Classification Labels", fontsize=14)
plt.xlabel("Classification Labels", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualization 2: T5 Self-Assessment Score Distribution
plt.figure(figsize=(8, 6))
data["T5 Self-Assessment"].replace("Invalid", 0, inplace=True)
data["T5 Self-Assessment"] = pd.to_numeric(data["T5 Self-Assessment"], errors="coerce")
data["T5 Self-Assessment"].plot(kind="hist", bins=10, color="orange", alpha=0.7)
plt.title("Distribution of T5 Self-Assessment Scores", fontsize=14)
plt.xlabel("Self-Assessment Score", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.tight_layout()
plt.show()

# Visualization 3: Context Requirement Breakdown
if "Needs Context?" in data.columns:
    plt.figure(figsize=(8, 6))
    context_needs = data["Needs Context?"].value_counts()
    context_needs.plot(kind="pie", autopct="%1.1f%%", startangle=140, colors=["gold", "lightgreen"])
    plt.title("Proportion of Questions Requiring Context", fontsize=14)
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

# Visualization 4: T5 Confidence vs. T5 Self-Assessment
data = clean_data(data, ["T5 Confidence", "T5 Self-Assessment"])
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x=data["T5 Confidence"],
    y=data["T5 Self-Assessment"],
    label="T5",
    color="blue",
    alpha=0.7,
    edgecolor="k"
)
sns.regplot(
    x=data["T5 Confidence"],
    y=data["T5 Self-Assessment"],
    scatter=False,
    color="blue",
    line_kws={"linestyle": "dashed"}
)
plt.title("T5 Confidence vs. T5 Self-Assessment", fontsize=14)
plt.xlabel("T5 Confidence (0-1)", fontsize=12)
plt.ylabel("T5 Self-Assessment Score", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Visualization 5: RoBERTa Confidence vs. RoBERTa Self-Assessment
data = clean_data(data, ["Roberta Confidence", "Roberta Self-Assessment"])
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x=data["Roberta Confidence"],
    y=data["Roberta Self-Assessment"],
    label="RoBERTa",
    color="red",
    alpha=0.7,
    edgecolor="black"
)

# Добавляем горизонтальную линию
plt.axhline(y=4, color="red", linestyle="dashed", linewidth=1.5, alpha=0.8)

# Настройки графика
plt.title("RoBERTa Confidence vs. RoBERTa Self-Assessment", fontsize=14)
plt.xlabel("RoBERTa Confidence (0-1)", fontsize=12)
plt.ylabel("RoBERTa Self-Assessment Score", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()

# Отображение
plt.show()

# Visualization 6: Comparison of T5 and RoBERTa Confidence vs. Self-Assessment
data = clean_data(data, ["T5 Confidence", "T5 Self-Assessment", "Roberta Confidence", "Roberta Self-Assessment"])
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=data["T5 Confidence"],
    y=data["T5 Self-Assessment"],
    label="T5",
    color="blue",
    alpha=0.7
)
sns.scatterplot(
    x=data["Roberta Confidence"],
    y=data["Roberta Self-Assessment"],
    label="RoBERTa",
    color="green",
    alpha=0.7
)
sns.regplot(
    x=data["T5 Confidence"],
    y=data["T5 Self-Assessment"],
    scatter=False,
    color="blue",
    line_kws={"linestyle": "dashed"}
)
sns.regplot(
    x=data["Roberta Confidence"],
    y=data["Roberta Self-Assessment"],
    scatter=False,
    color="green",
    line_kws={"linestyle": "dashed"}
)
plt.title("Comparison of T5 and RoBERTa: Confidence vs. Self-Assessment", fontsize=16, weight="bold")
plt.xlabel("Confidence (0-1)", fontsize=14)
plt.ylabel("Self-Assessment Score", fontsize=14)
plt.legend(title="Model", fontsize=12, loc="upper left")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Install necessary libraries
!pip install --quiet matplotlib pandas gspread oauth2client seaborn scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from google.colab import auth
import gspread
from google.auth import default

# ---------------------------
# 1) Authenticate and Load Data
# ---------------------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Replace with your Google Sheet URL
sheet = gc.open_by_url(sheet_url).sheet1

# Load data into a Pandas DataFrame
data = pd.DataFrame(sheet.get_all_records())

# ---------------------------
# 2) Data Preparation
# ---------------------------
# Ensure numeric conversion where needed
data["Roberta Confidence"] = pd.to_numeric(data["Roberta Confidence"], errors="coerce")
data = data.dropna(subset=["Classification", "Original Label"])

# Map labels to numerical values for metrics calculation
label_mapping = {"SUPPORTS": 0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
data["Predicted"] = data["Classification"].map(label_mapping)
data["Actual"] = data["Original Label"].map(label_mapping)

# Drop rows with unmapped labels
filtered_data = data.dropna(subset=["Predicted", "Actual"])

# Extract predictions and ground truth
predictions = filtered_data["Predicted"].astype(int)
ground_truth = filtered_data["Actual"].astype(int)

# ---------------------------
# 3) Visualizations
# ---------------------------

# Visualization 1: Confusion Matrix
conf_matrix = confusion_matrix(ground_truth, predictions, labels=[0, 1, 2])
ConfusionMatrixDisplay(conf_matrix, display_labels=label_mapping.keys()).plot(cmap="Blues")
plt.title("Confusion Matrix: RoBERTa Predictions vs. Ground Truth", fontsize=14, weight="bold")
plt.show()

# Visualization 2: Classification Report
print("Classification Report:\n")
print(classification_report(ground_truth, predictions, target_names=label_mapping.keys()))

# Visualization 3: Accuracy per Class
class_counts = filtered_data.groupby("Original Label").size()
accurate_counts = filtered_data[filtered_data["Predicted"] == filtered_data["Actual"]].groupby("Original Label").size()

accuracy_per_class = (accurate_counts / class_counts * 100).reindex(label_mapping.keys(), fill_value=0)
plt.figure(figsize=(8, 6))
accuracy_per_class.plot(kind="bar", color=["skyblue", "salmon", "lime"], alpha=0.8)
plt.title("Accuracy per Class", fontsize=14, weight="bold")
plt.xlabel("Class", fontsize=12)
plt.ylabel("Accuracy (%)", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Visualization 4: Confidence Distribution for Correct/Incorrect Predictions
filtered_data["Correct"] = filtered_data["Predicted"] == filtered_data["Actual"]
plt.figure(figsize=(10, 6))
sns.histplot(data=filtered_data, x="Roberta Confidence", hue="Correct", kde=True, palette={True: "green", False: "red"}, alpha=0.6)
plt.title("Confidence Distribution: Correct vs. Incorrect Predictions", fontsize=14, weight="bold")
plt.xlabel("Confidence (0-1)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Visualization 5: Distribution of Dataset Labels
plt.figure(figsize=(8, 6))
data["Original Label"].value_counts().plot(kind="bar", color=["skyblue", "salmon", "lime"], alpha=0.8)
plt.title("Distribution of Dataset Labels", fontsize=14, weight="bold")
plt.xlabel("Labels", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()
